#### [Naver 리뷰 데이터셋 전처리]
<hr>

- train :  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
- test :  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt

In [1]:
# 데이터 다운로드
from urllib.request import urlretrieve

filename = '../DATA/ratings_train.txt'

urlretrieve('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', filename)

filename = '../DATA/ratings_test.txt'

urlretrieve('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', filename)


('../DATA/ratings_test.txt', <http.client.HTTPMessage at 0x7022bcc89340>)

In [2]:
# 데이터 로드

import pandas as pd

train_data = pd.read_csv('../DATA/ratings_train.txt', delimiter='\t')
test_data = pd.read_csv('../DATA/ratings_test.txt', delimiter='\t')
train_data.head(), test_data.head()

(         id                                           document  label
 0   9976970                                아 더빙.. 진짜 짜증나네요 목소리      0
 1   3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
 2  10265843                                  너무재밓었다그래서보는것을추천한다      0
 3   9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
 4   6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1,
         id                                           document  label
 0  6270596                                                굳 ㅋ      1
 1  9274899                               GDNTOPCLASSINTHECLUB      0
 2  8544678             뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아      0
 3  6825595                   지루하지는 않은데 완전 막장임... 돈주고 보기에는....      0
 4  6723715  3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??      0)

#### (2) 데이터 전처리
<hr>

- 결측치, 중복데이터
- 클래스의 균형 데이터 여부
- 텍스트 데이터 전처리 => 정제 (불용어, 노이즈데이터 제거), 토큰화, 단어사전
- 텍스트 데이터 인코딩
- 텍스트 데이터 패딩


In [3]:
train_data.label.value_counts()

0    75173
1    74827
Name: label, dtype: int64

In [4]:
import konlpy.tag as Tag
tagger = Tag.Mecab()



In [5]:

morphs0 = [tagger.nouns(x) for x in train_data[train_data['label'] == 0].document.to_list() if isinstance(x, str)]
morphs1 = [tagger.nouns(x) for x in train_data[train_data['label'] == 1].document.to_list() if isinstance(x, str)]
pos0 = [tagger.pos(x) for x in train_data[train_data['label'] == 0].document.to_list() if isinstance(x, str)]
pos1 = [tagger.pos(x) for x in train_data[train_data['label'] == 1].document.to_list() if isinstance(x, str)]

result0 = []
result1 = []
resultpos0 = []
resultpos1 = []
def oneD(morphs0, result0):
    for idx, x in enumerate(morphs0):
        result0 += x

oneD(morphs0, result0)
oneD(morphs1, result1)

oneD(pos0, resultpos0)
oneD(pos1, resultpos1)


In [6]:
resultpos0

[('아', 'IC'),
 ('더', 'MAG'),
 ('빙', 'MAG'),
 ('.', 'SF'),
 ('.', 'SY'),
 ('진짜', 'MAG'),
 ('짜증', 'NNG'),
 ('나', 'NP'),
 ('네요', 'VCP+EC'),
 ('목소리', 'NNG'),
 ('너무', 'MAG'),
 ('재', 'XPN'),
 ('밓었다그래서보는것을추천한다', 'UNKNOWN'),
 ('교도소', 'NNG'),
 ('이야기', 'NNG'),
 ('구먼', 'VCP+EF'),
 ('.', 'SF'),
 ('.', 'SY'),
 ('솔직히', 'MAG'),
 ('재미', 'NNG'),
 ('는', 'JX'),
 ('없', 'VA'),
 ('다', 'EF'),
 ('.', 'SF'),
 ('.', 'SY'),
 ('평점', 'NNG'),
 ('조정', 'NNG'),
 ('막', 'MAJ'),
 ('걸음마', 'NNG'),
 ('뗀', 'VV+ETM'),
 ('3', 'SN'),
 ('세', 'NNBC'),
 ('부터', 'JX'),
 ('초등', 'NNG'),
 ('학교', 'NNG'),
 ('1', 'SN'),
 ('학년', 'NNG'),
 ('생', 'XSN'),
 ('인', 'VCP+ETM'),
 ('8', 'SN'),
 ('살', 'NNBC'),
 ('용', 'XSN'),
 ('영화', 'NNG'),
 ('.', 'SF'),
 ('ㅋㅋㅋ', 'IC'),
 ('.', 'SF'),
 ('..', 'SY'),
 ('별반', 'MAG'),
 ('개', 'NNG'),
 ('도', 'JX'),
 ('아까움', 'VA+ETN'),
 ('.', 'SF'),
 ('원작', 'NNG'),
 ('의', 'JKG'),
 ('긴장감', 'NNG'),
 ('을', 'JKO'),
 ('제대로', 'MAG'),
 ('살려', 'VV+EC'),
 ('내', 'VX'),
 ('지', 'EC'),
 ('못했', 'VX+EP'),
 ('다', 'EF'),
 ('.', 'SF'),
 ('별'

In [7]:
dfpos0 = pd.DataFrame(resultpos0)
dfpos0.columns = ['word', 'pos']
dfpos0[dfpos0['pos'].str.startswith('V')]

,word,pos
8,네요,VCP+EC
15,구먼,VCP+EF
21,없,VA
29,뗀,VV+ETM
38,인,VCP+ETM
...,...,...
1408663,지,VCP+EF
1408670,인가,VCP+EF
1408678,거들먹거리,VV
1408683,착하,VA


In [8]:
pd.Series(result0).to_csv('../DATA/bad_review.csv', index=False, header=False)
pd.Series(result1).to_csv('../DATA/good_review.csv', index=False, header=False)

In [9]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.normalizers import Lowercase, Sequence, NFD
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.decoders import WordPiece as WordPieceDecoder

tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
tokenizer.normalizer = Sequence([NFD(), Lowercase()])
tokenizer.pre_tokenizer = Whitespace()

tokenizer.train(['../DATA/bad_review.csv'])
tokenizer.save('../DATA/bad_review.json')

tokenizer.train(['../DATA/good_review.csv'])
tokenizer.save('../DATA/good_review.json')